In [1]:
"""
Surgery Scheduling Optimization with Overtime Penalties
========================================================

This model schedules surgeries across operating rooms with stochastic durations,
using SOFT capacity constraints with overtime penalties instead of hard caps.

Key Changes from Original:
- Room capacity can be exceeded by paying overtime costs
- Doctor capacity can be exceeded by paying overtime costs
- Hard caps on maximum overtime (e.g., 120 min per room)
- Overtime costs added to objective function
- Reliability constraints still use regular hours (not OT)

FIXES APPLIED (based on ChatGPT feedback):
✓ Fix 1: Replaced Python max() in objective with explicit Idle variables + constraints
✓ Fix 2: Bind doctor start times to OT_doc[d,k] variable (not constant)
✓ Fix 3: Bind room start times to OT_room[d,r] variable (not constant)
✓ Fix 4: Use Idle variables in objective function
"""

import os
import pyomo.environ as pyo
from pyomo.opt import TerminationCondition
import math
import json
import time
from datetime import datetime

# CRITICAL: Set up Gurobi WLS license BEFORE any Pyomo operations
license_paths = [
    os.path.expanduser('~/.gurobi/gurobi.lic'),
    os.path.expanduser('~/gurobi.lic'),
    '/opt/gurobi/gurobi.lic'
]

#for path in license_paths:
#    if os.path.exists(path):
#        os.environ['GRB_LICENSE_FILE'] = path
#        print(f"✓ Using Gurobi license file: {path}")
#        break
#else:
#    print("⚠ Warning: No Gurobi license file found. Will use restricted license.")


def load_instance(filename):
    """Load problem instance from JSON file."""
    with open(filename, 'r') as f:
        return json.load(f)


def precompute_durations(instance, alpha_choices):
    """
    Precompute buffered durations for all valid (surgery, room, doctor, alpha) combinations.
    Using Cantelli's inequality for buffer calculation.
    
    CORRECT Cantelli formula: δ = μ + σ * sqrt((1-α)/α)
    """
    durations = {}
    
    for surgery in instance['surgeries']:
        j = surgery['id']
        durations[j] = {}
        
        for room in instance['rooms']:
            r = room['id']
            durations[j][r] = {}
            
            for doctor in instance['doctors']:
                k = doctor['id']
                durations[j][r][k] = {}
                
                # Check format and get mu, sigma
                mu, sigma = None, None
                
                # Format 1: mu_sigma dictionary (no specialization)
                if 'mu_sigma' in surgery:
                    key = f"{r}|{k}"
                    if key in surgery['mu_sigma']:
                        mu = surgery['mu_sigma'][key]['mu']
                        sigma = surgery['mu_sigma'][key]['sigma']
                
                # Format 2: duration_mean/std (with specialization)
                elif 'duration_mean' in surgery and 'duration_std' in surgery:
                    # Check compatibility
                    surgery_specialty = surgery.get('specialty')
                    doctor_specialties = doctor.get('specialties', [])
                    room_types = room.get('types', [])
                    
                    # Skip if incompatible
                    if surgery_specialty:
                        if surgery_specialty not in doctor_specialties:
                            continue
                        if surgery_specialty not in room_types:
                            continue
                    
                    mu = surgery['duration_mean']
                    sigma = surgery['duration_std']
                
                # If we have mu and sigma, compute buffered durations
                if mu is not None and sigma is not None:
                    for alpha_idx, alpha in enumerate(alpha_choices):
                        if alpha >= 1.0:
                            buffer_duration = float('inf')
                        else:
                            # CORRECT Cantelli formula: sqrt((1-α)/α)
                            buffer = sigma * math.sqrt((1 - alpha) / alpha)
                            buffer_duration = mu + buffer
                        
                        durations[j][r][k][alpha_idx] = buffer_duration
    
    return durations


def build_model_with_overtime(instance, alpha_choices, epsilon_by_day, 
                               ot_cost_room=3.0, ot_cost_doc=1.5,
                               max_ot_room=120, max_ot_doc=60):
    """
    Build the optimization model with SOFT capacity constraints (overtime).
    
    Key Changes:
    - Room and doctor capacity constraints now allow overtime
    - Overtime variables OT[d,r] and OT_doc[d,k] added
    - Hard caps on overtime (max_ot_room, max_ot_doc)
    - Overtime costs added to objective
    - Reliability constraints still use regular hours Hd (not Hd+OT)
    
    Parameters:
    -----------
    instance : dict
        Problem instance data
    alpha_choices : tuple
        Discretized alpha values to choose from
    epsilon_by_day : dict or float
        Fixed epsilon value(s)
    ot_cost_room : float
        Cost per minute of room overtime (€/min)
    ot_cost_doc : float
        Cost per minute of doctor overtime (€/min)
    max_ot_room : float
        Hard cap on room overtime per day (minutes)
    max_ot_doc : float
        Hard cap on doctor overtime per day (minutes)
    
    Returns:
    --------
    model, durations, surg_dict, day_dict, doc_dict, alpha_choices
    """
    print("Precomputing buffered durations...")
    durations = precompute_durations(instance, alpha_choices)
    
    # Convert epsilon to per-day dict if needed
    if isinstance(epsilon_by_day, (int, float)):
        epsilon_dict = {d['id']: float(epsilon_by_day) for d in instance['days']}
    else:
        epsilon_dict = {}
        for d in instance['days']:
            day_id = d['id']
            if day_id in epsilon_by_day:
                epsilon_dict[day_id] = float(epsilon_by_day[day_id])
            elif str(day_id) in epsilon_by_day:
                epsilon_dict[day_id] = float(epsilon_by_day[str(day_id)])
            else:
                epsilon_dict[day_id] = 0.25
    
    print(f"Using epsilon values: {epsilon_dict}")
    print(f"\n💰 OVERTIME PARAMETERS:")
    print(f"  • Room overtime cost: €{ot_cost_room:.2f}/min")
    print(f"  • Doctor overtime cost: €{ot_cost_doc:.2f}/min")
    print(f"  • Max room OT per day: {max_ot_room} minutes")
    print(f"  • Max doctor OT per day: {max_ot_doc} minutes")
    
    # Create lookup dictionaries
    surgeries, days, rooms, doctors = instance["surgeries"], instance["days"], instance["rooms"], instance["doctors"]
    J = [s["id"] for s in surgeries]
    D = [d["id"] for d in days]
    R = [r["id"] for r in rooms]
    K = [k["id"] for k in doctors]
    T = list(range(len(alpha_choices)))  # Alpha indices
    
    surg_dict = {s['id']: s for s in surgeries}
    day_dict = {d['id']: d for d in days}
    doc_dict = {k['id']: k for k in doctors}
    
    # Build valid combinations
    valid = set()
    for j in J:
        for d in D:
            for r in R:
                for k in K:
                    # Check if doctor has capacity on this day
                    if k in doc_dict:
                        doctor_capacity = doc_dict[k].get('daily_capacity', {})
                        if isinstance(doctor_capacity, dict):
                            cap = doctor_capacity.get(d, 0)
                        else:
                            cap = day_dict[d]['H']
                    else:
                        cap = day_dict[d]['H']
                    
                    if cap > 0:
                        for t_idx in T:
                            if durations[j][r][k].get(t_idx) is not None:
                                valid.add((j, d, r, k, t_idx))
    
    print(f"Valid combinations: {len(valid):,}")
    
    # Identify which doctors can do which surgeries (for pruning)
    surgery_doctors = {}
    for j in J:
        surgery_doctors[j] = set()
        for (j2, d, r, k, t) in valid:
            if j2 == j:
                surgery_doctors[j].add(k)
    
    # Create model
    model = pyo.ConcreteModel()
    model.J = pyo.Set(initialize=J)
    model.D = pyo.Set(initialize=D)
    model.R = pyo.Set(initialize=R)
    model.K = pyo.Set(initialize=K)
    model.T = pyo.Set(initialize=T)
    model.VALID = pyo.Set(initialize=valid, dimen=5)
    
    # Parameters for reliability constraint (log form)
    model.epsilon_param = pyo.Param(model.D, initialize=epsilon_dict, mutable=True)
    model.log_one_minus_eps = pyo.Param(model.D, 
                                        initialize={d: math.log(1 - epsilon_dict[d]) for d in D},
                                        mutable=True)
    
    # ========================================
    # DECISION VARIABLES
    # ========================================
    
    # 1. Start times in rooms
    model.s = pyo.Var(model.J, model.D, model.R, 
                     bounds=lambda m,j,d,r: (0.0, day_dict[d]["H"] + max_ot_room),  # Can go into OT
                     domain=pyo.NonNegativeReals)
    
    # 2. Binary assignment variables
    model.w = pyo.Var(model.VALID, domain=pyo.Binary)
    
    # 3. Room sequencing variables
    print(f"  Creating room sequencing variables...")
    model.u_room = pyo.Var([(j, i, d, r) for j in J for i in J for d in D for r in R if j < i],
                          domain=pyo.Binary)
    
    # 4. Doctor sequencing variables (with pruning)
    print(f"  Creating doctor sequencing variables (with pruning)...")
    pruned_u_doc = []
    for j in J:
        for i in J:
            if j < i:
                shared = surgery_doctors[j] & surgery_doctors[i]
                for d in D:
                    for k in shared:
                        pruned_u_doc.append((j, i, d, k))
    
    model.u_doc = pyo.Var(pruned_u_doc, domain=pyo.Binary)
    
    original = len(J)*(len(J)-1)//2 * len(D) * len(K)
    pruning = 100 * (1 - len(pruned_u_doc)/original) if original > 0 else 0
    print(f"  u_doc variables: {original:,} → {len(pruned_u_doc):,} ({pruning:.1f}% reduction)")
    
    # 5. Doctor start times and durations
    model.s_doc = pyo.Var(model.J, model.D, model.K, domain=pyo.NonNegativeReals)
    model.dur_doc = pyo.Var(model.J, model.D, model.K, domain=pyo.NonNegativeReals)
    
    # ========================================
    # NEW: OVERTIME VARIABLES
    # ========================================
    print(f"  Creating overtime variables...")
    
    # Room overtime per day
    model.OT_room = pyo.Var(model.D, model.R, 
                           bounds=(0.0, max_ot_room),
                           domain=pyo.NonNegativeReals)
    
    # Doctor overtime per day
    model.OT_doc = pyo.Var(model.D, model.K, 
                          bounds=(0.0, max_ot_doc),
                          domain=pyo.NonNegativeReals)
    
    # NEW: Idle time variables (to avoid Python max() in objective)
    model.Idle = pyo.Var(model.D, model.R, domain=pyo.NonNegativeReals)
    
    # Expression: y[j,d,k] = 1 if surgery j assigned to doctor k on day d
    model.y = pyo.Expression(model.J, model.D, model.K,
                            rule=lambda m,j,d,k: sum(m.w[j,d,r,k,t] 
                                                    for (j2,d2,r,k2,t) in m.VALID 
                                                    if j2==j and d2==d and k2==k))
    
    # Big-M values
    max_dur = 0
    for j in J:
        for r in R:
            for k in K:
                if durations[j][r][k]:
                    max_dur = max(max_dur, max(durations[j][r][k].values()))
    M_room = {d: day_dict[d]["H"] + max_ot_room + max_dur for d in D}
    
    # ========================================
    # CONSTRAINTS
    # ========================================
    
    # C1: Each surgery scheduled exactly once
    def assign_rule(m, j):
        return sum(m.w[j,d,r,k,t] for (j2,d,r,k,t) in m.VALID if j2==j) == 1
    model.assign = pyo.Constraint(model.J, rule=assign_rule)
    
    # C2: Room capacity constraints (MODIFIED - now with overtime)
    def room_cap_rule(m, j, d, r):
        chosen = sum(m.w[j,d,r,k,t] for (j2,d2,r2,k,t) in m.VALID 
                    if j2==j and d2==d and r2==r)
        dur_sum = sum(durations[j][r][k][t] * m.w[j,d,r,k,t] 
                     for (j2,d2,r2,k,t) in m.VALID 
                     if j2==j and d2==d and r2==r and durations[j][r][k].get(t) is not None)
        # CHANGE: Allow exceeding Hd by using OT_room
        return m.s[j,d,r] + dur_sum <= day_dict[d]["H"] + m.OT_room[d,r] + M_room[d] * (1 - chosen)
    model.room_cap = pyo.Constraint(model.J, model.D, model.R, rule=room_cap_rule)
    
    # C3: Doctor capacity constraints (MODIFIED - now with overtime)
    def doc_cap_rule(m, k, d):
        if k in doc_dict:
            doctor_capacity = doc_dict[k].get('daily_capacity', {})
            if isinstance(doctor_capacity, dict):
                cap = doctor_capacity.get(d, 0)
            else:
                cap = day_dict[d]['H']
        else:
            cap = day_dict[d]['H']
        
        if cap == 0:
            return pyo.Constraint.Skip
        
        # CHANGE: Allow exceeding cap by using OT_doc
        return sum(durations[j][r][k][t] * m.w[j,d,r,k,t] 
                  for (j,d2,r,k2,t) in m.VALID 
                  if k2==k and d2==d and durations[j][r][k].get(t) is not None) <= cap + m.OT_doc[d,k]
    model.doc_cap = pyo.Constraint(model.K, model.D, rule=doc_cap_rule)
    
    # C4-C5: Room-doctor start time synchronization
    def sdoc_upper_rule(m, j, d, k, r):
        return m.s_doc[j,d,k] <= m.s[j,d,r] + M_room[d] * (1 - sum(m.w[j,d,r,k,t] 
                                                                    for t in m.T 
                                                                    if (j,d,r,k,t) in m.VALID))
    model.sdoc_upper = pyo.Constraint(model.J, model.D, model.K, model.R, rule=sdoc_upper_rule)
    
    def sdoc_lower_rule(m, j, d, k, r):
        return m.s_doc[j,d,k] >= m.s[j,d,r] - M_room[d] * (1 - sum(m.w[j,d,r,k,t] 
                                                                    for t in m.T 
                                                                    if (j,d,r,k,t) in m.VALID))
    model.sdoc_lower = pyo.Constraint(model.J, model.D, model.K, model.R, rule=sdoc_lower_rule)
    
    # C6: Doctor start time upper bound (FIXED: bind to OT_doc variable)
    def sdoc_upper_H_rule(m, j, d, k):
        return m.s_doc[j,d,k] <= day_dict[d]["H"] + m.OT_doc[d,k] + M_room[d] * (1 - m.y[j,d,k])
    model.sdoc_upper_H = pyo.Constraint(model.J, model.D, model.K, rule=sdoc_upper_H_rule)
    
    # C6b: Room start time upper bound (NEW: bind to OT_room variable)
    def s_room_upper_rule(m, j, d, r):
        chosen = sum(m.w[j,d,r,k,t] for (j2,d2,r2,k,t) in m.VALID
                    if j2==j and d2==d and r2==r)
        return m.s[j,d,r] <= day_dict[d]["H"] + m.OT_room[d,r] + M_room[d] * (1 - chosen)
    model.s_room_upper = pyo.Constraint(model.J, model.D, model.R, rule=s_room_upper_rule)
    
    # C7: Link doctor duration to chosen alpha
    def durdoc_link_rule(m, j, d, k):
        return m.dur_doc[j,d,k] == sum(durations[j][r][k][t] * m.w[j,d,r,k,t] 
                                       for (j2,d2,r,k2,t) in m.VALID 
                                       if j2==j and d2==d and k2==k and durations[j][r][k].get(t) is not None)
    model.durdoc_link = pyo.Constraint(model.J, model.D, model.K, rule=durdoc_link_rule)
    
    # C7b: Link idle time variables (FIX: avoid Python max() in objective)
    def idle_link_rule(m, d, r):
        used = sum(durations[j][r][k][t] * m.w[j,d,r,k,t]
                  for (j,d2,r2,k,t) in m.VALID
                  if d2==d and r2==r and durations[j][r][k].get(t) is not None)
        return m.Idle[d,r] >= day_dict[d]["H"] - used
    model.idle_link = pyo.Constraint(model.D, model.R, rule=idle_link_rule)
    
    # C8-C9: Room sequencing (non-overlap)
    def room_seq_fwd_rule(m, j, i, d, r):
        if (j,i,d,r) not in m.u_room.index_set():
            return pyo.Constraint.Skip
        
        chosen_j = sum(m.w[j,d,r,k,t] for (j2,d2,r2,k,t) in m.VALID 
                      if j2==j and d2==d and r2==r)
        chosen_i = sum(m.w[i,d,r,k,t] for (i2,d2,r2,k,t) in m.VALID 
                      if i2==i and d2==d and r2==r)
        dur_j = sum(durations[j][r][k][t] * m.w[j,d,r,k,t] 
                   for (j2,d2,r2,k,t) in m.VALID 
                   if j2==j and d2==d and r2==r and durations[j][r][k].get(t) is not None)
        
        return m.s[i,d,r] >= m.s[j,d,r] + dur_j - M_room[d] * (2 - chosen_j - chosen_i) - M_room[d] * (1 - m.u_room[j,i,d,r])
    model.room_seq_fwd = pyo.Constraint([(j,i,d,r) for j in J for i in J for d in D for r in R if j<i],
                                       rule=room_seq_fwd_rule)
    
    def room_seq_bwd_rule(m, j, i, d, r):
        if (j,i,d,r) not in m.u_room.index_set():
            return pyo.Constraint.Skip
        
        chosen_j = sum(m.w[j,d,r,k,t] for (j2,d2,r2,k,t) in m.VALID 
                      if j2==j and d2==d and r2==r)
        chosen_i = sum(m.w[i,d,r,k,t] for (i2,d2,r2,k,t) in m.VALID 
                      if i2==i and d2==d and r2==r)
        dur_i = sum(durations[i][r][k][t] * m.w[i,d,r,k,t] 
                   for (i2,d2,r2,k,t) in m.VALID 
                   if i2==i and d2==d and r2==r and durations[i][r][k].get(t) is not None)
        
        return m.s[j,d,r] >= m.s[i,d,r] + dur_i - M_room[d] * (2 - chosen_j - chosen_i) - M_room[d] * m.u_room[j,i,d,r]
    model.room_seq_bwd = pyo.Constraint([(j,i,d,r) for j in J for i in J for d in D for r in R if j<i],
                                       rule=room_seq_bwd_rule)
    
    # C10-C11: Doctor sequencing (non-overlap)
    def doctor_seq_fwd_rule(m, j, i, d, k):
        if (j,i,d,k) not in m.u_doc.index_set():
            return pyo.Constraint.Skip
        return m.s_doc[i,d,k] >= m.s_doc[j,d,k] + m.dur_doc[j,d,k] - M_room[d] * (2 - m.y[j,d,k] - m.y[i,d,k]) - M_room[d] * (1 - m.u_doc[j,i,d,k])
    model.doctor_seq_fwd = pyo.Constraint(pruned_u_doc, rule=doctor_seq_fwd_rule)
    
    def doctor_seq_bwd_rule(m, j, i, d, k):
        if (j,i,d,k) not in m.u_doc.index_set():
            return pyo.Constraint.Skip
        return m.s_doc[j,d,k] >= m.s_doc[i,d,k] + m.dur_doc[i,d,k] - M_room[d] * (2 - m.y[j,d,k] - m.y[i,d,k]) - M_room[d] * m.u_doc[j,i,d,k]
    model.doctor_seq_bwd = pyo.Constraint(pruned_u_doc, rule=doctor_seq_bwd_rule)
    
    # C12: RELIABILITY CONSTRAINT (UNCHANGED - still uses regular Hd, not Hd+OT)
    ln_alpha = [math.log(1 - a) for a in alpha_choices]
    
    def reliability_rule(m, d):
        return sum(ln_alpha[t] * m.w[j,d,r,k,t] 
                  for (j,d2,r,k,t) in m.VALID if d2==d) >= m.log_one_minus_eps[d]
    model.reliability = pyo.Constraint(model.D, rule=reliability_rule)
    
    # ========================================
    # OBJECTIVE: Minimize idle time + OVERTIME COSTS
    # ========================================
    def obj_rule(m):
        # FIXED: Use Idle variables (not Python max())
        total_idle = sum(m.Idle[d,r] for d in D for r in R)
        
        # Overtime costs
        room_ot_cost = ot_cost_room * sum(m.OT_room[d,r] for d in D for r in R)
        doc_ot_cost = ot_cost_doc * sum(m.OT_doc[d,k] for d in D for k in K)
        
        # Small penalty for late starts (tie-breaker)
        start_penalty = 0.001 * sum(m.s[j,d,r] for j in J for d in D for r in R)
        
        return total_idle + room_ot_cost + doc_ot_cost + start_penalty
    
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)
    
    # Print statistics
    num_u_room = len([1 for j in J for i in J for d in D for r in R if j<i])
    print(f"\n✓ Model with OVERTIME built")
    print(f"\nModel statistics:")
    print(f"  Surgeries: {len(J)}")
    print(f"  Alpha choices: {len(alpha_choices)}")
    print(f"  Binary w variables: {len(valid):,}")
    print(f"  Binary u_room variables: {num_u_room:,}")
    print(f"  Binary u_doc variables: {len(pruned_u_doc):,}")
    print(f"  Continuous start time variables: {len(J)*len(D)*len(R):,}")
    print(f"  Overtime variables: {len(D)*len(R) + len(D)*len(K):,} (room + doctor)")
    print(f"  Idle time variables: {len(D)*len(R):,}")
    print(f"  Total binary variables: {len(valid) + num_u_room + len(pruned_u_doc):,}")
    
    return model, durations, surg_dict, day_dict, doc_dict, alpha_choices


def solve_model(model, time_limit=900, mip_gap=0.15):
    """Solve the model using Gurobi with persistent interface (WSL compatible)."""
    solver_names = ['gurobi_persistent', 'gurobi_direct', 'gurobi']
    solver = None
    solver_name_used = None
    
    for solver_name in solver_names:
        try:
            solver = pyo.SolverFactory(solver_name)
            if solver.available():
                solver_name_used = solver_name
                print(f"Using solver: {solver_name}")
                break
        except:
            continue
    
    if solver is None or not solver.available():
        raise RuntimeError("No Gurobi solver available. Please check your Gurobi installation and license.")
    
    # Set solver options
    if solver_name_used == 'gurobi_persistent':
        solver.set_instance(model)
        solver.set_gurobi_param('TimeLimit', time_limit)
        solver.set_gurobi_param('MIPGap', mip_gap)
        solver.set_gurobi_param('Threads', 0)
        solver.set_gurobi_param('MIPFocus', 1)
        
        print(f"Solving with {solver_name_used} (TimeLimit={time_limit}s, MIPGap={mip_gap})...")
        start_time = time.time()
        results = solver.solve(tee=True, save_results=False)
        solve_time = time.time() - start_time
    else:
        solver.options['TimeLimit'] = time_limit
        solver.options['MIPGap'] = mip_gap
        solver.options['Threads'] = 0
        solver.options['MIPFocus'] = 1
        
        print(f"Solving with {solver_name_used} (TimeLimit={time_limit}s, MIPGap={mip_gap})...")
        start_time = time.time()
        results = solver.solve(model, tee=True)
        solve_time = time.time() - start_time
    
    print(f"\nSolve completed in {solve_time:.1f} seconds ({solve_time/60:.1f} minutes)")
    
    return results, solve_time


def extract_schedule_with_overtime(model, instance, durations, alpha_choices):
    """Extract the schedule with overtime information."""
    schedule = []
    
    for (j, t, r, k, alpha_idx) in model.VALID:
        if pyo.value(model.w[j, t, r, k, alpha_idx]) > 0.5:
            surgery = next((s for s in instance['surgeries'] if s['id'] == j), None)
            day = next((d for d in instance['days'] if d['id'] == t), None)
            room = next((rm for rm in instance['rooms'] if rm['id'] == r), None)
            doctor = next((doc for doc in instance['doctors'] if doc['id'] == k), None)
            
            if surgery and day and room and doctor:
                start_time = pyo.value(model.s[j, t, r])
                
                if 'mu_sigma' in surgery:
                    key = f"{r}|{k}"
                    if key in surgery['mu_sigma']:
                        duration_mean = surgery['mu_sigma'][key]['mu']
                        duration_std = surgery['mu_sigma'][key]['sigma']
                    else:
                        duration_mean = surgery.get('mu', 0)
                        duration_std = surgery.get('sigma', 0)
                else:
                    duration_mean = surgery.get('duration_mean', surgery.get('mu', 0))
                    duration_std = surgery.get('duration_std', surgery.get('sigma', 0))
                
                buffered_dur = durations[j][r][k][alpha_idx]
                end_time = start_time + buffered_dur
                
                surgery_name = surgery.get('name', str(j))
                day_name = day.get('name', str(t))
                room_name = room.get('name', str(r))
                doctor_name = doctor.get('name', str(k))
                specialty = surgery.get('specialty', 'General')
                
                schedule.append({
                    'surgery_id': j,
                    'surgery_name': surgery_name,
                    'specialty': specialty,
                    'day': t,
                    'day_name': day_name,
                    'room': r,
                    'room_name': room_name,
                    'doctor': k,
                    'doctor_name': doctor_name,
                    'alpha': alpha_choices[alpha_idx],
                    'duration_mean': duration_mean,
                    'duration_std': duration_std,
                    'buffered_duration': buffered_dur,
                    'start': start_time,
                    'end': end_time
                })
    
    # Extract overtime info
    overtime_info = {
        'room': {},
        'doctor': {}
    }
    
    for d in model.D:
        for r in model.R:
            ot = pyo.value(model.OT_room[d, r])
            if ot > 0.01:  # Only record if >0
                if d not in overtime_info['room']:
                    overtime_info['room'][d] = {}
                overtime_info['room'][d][r] = ot
        
        for k in model.K:
            ot = pyo.value(model.OT_doc[d, k])
            if ot > 0.01:
                if d not in overtime_info['doctor']:
                    overtime_info['doctor'][d] = {}
                overtime_info['doctor'][d][k] = ot
    
    # Compute idle time and overtime costs
    total_capacity = sum(d['H'] for d in instance['days']) * len(instance['rooms'])
    used_time = sum(s['buffered_duration'] for s in schedule)
    
    # Idle time only in regular hours
    total_idle = 0
    for d in instance['days']:
        for r in instance['rooms']:
            day_room_used = sum(s['buffered_duration'] for s in schedule 
                               if s['day'] == d['id'] and s['room'] == r['id'])
            total_idle += max(0, d['H'] - day_room_used)
    
    # Sort schedule
    day_order = [d['id'] for d in instance['days']]
    day_to_idx = {d: idx for idx, d in enumerate(day_order)}
    schedule.sort(key=lambda x: (day_to_idx.get(x['day'], 999), x['room'], x['start']))
    
    return schedule, total_idle, overtime_info


def print_schedule_with_overtime(schedule, instance, epsilon_dict, overtime_info, ot_cost_room, ot_cost_doc):
    """Print detailed schedule including overtime information."""
    print("\n" + "="*100)
    print("📋 DETAILED SCHEDULE WITH OVERTIME")
    print("="*100)
    
    by_day = {}
    for s in schedule:
        if s['day'] not in by_day:
            by_day[s['day']] = []
        by_day[s['day']].append(s)
    
    day_order = [d['id'] for d in instance['days']]
    
    for day_id in day_order:
        if day_id not in by_day:
            continue
        
        day = next(d for d in instance['days'] if d['id'] == day_id)
        surgeries = by_day[day_id]
        
        if isinstance(epsilon_dict, dict):
            epsilon_val = epsilon_dict.get(day_id, 'N/A')
        else:
            epsilon_val = epsilon_dict
        
        day_display = day.get('name', str(day_id))
        
        # Get overtime for this day
        day_room_ot = overtime_info['room'].get(day_id, {})
        day_doc_ot = overtime_info['doctor'].get(day_id, {})
        total_day_room_ot = sum(day_room_ot.values())
        total_day_doc_ot = sum(day_doc_ot.values())
        
        print(f"\n{'─'*100}")
        print(f"Day: {day_display} (Regular Hours: {day['H']} min, ε = {epsilon_val:.4f})")
        if total_day_room_ot > 0 or total_day_doc_ot > 0:
            print(f"  💰 OVERTIME: Room={total_day_room_ot:.1f} min (€{total_day_room_ot*ot_cost_room:.2f}), "
                  f"Doctor={total_day_doc_ot:.1f} min (€{total_day_doc_ot*ot_cost_doc:.2f})")
        print(f"{'─'*100}")
        
        by_room = {}
        for s in surgeries:
            if s['room'] not in by_room:
                by_room[s['room']] = []
            by_room[s['room']].append(s)
        
        for room_id in sorted(by_room.keys()):
            room_surgeries = by_room[room_id]
            room = next((r for r in instance['rooms'] if r['id'] == room_id), None)
            room_display = room.get('name', str(room_id)) if room else str(room_id)
            
            total_time = sum(s['buffered_duration'] for s in room_surgeries)
            regular_hours = day['H']
            room_ot = day_room_ot.get(room_id, 0)
            
            if total_time > regular_hours:
                utilization_regular = 100.0
                overtime_used = total_time - regular_hours
                print(f"\n  Room {room_display} - {len(room_surgeries)} surgeries, "
                      f"{total_time:.0f} min total ({regular_hours:.0f} regular + {overtime_used:.0f} OT)")
                print(f"    ⚠️ Using {room_ot:.1f} min overtime (€{room_ot*ot_cost_room:.2f})")
            else:
                utilization_regular = 100 * total_time / regular_hours
                print(f"\n  Room {room_display} - {len(room_surgeries)} surgeries, "
                      f"{total_time:.0f}/{regular_hours} min ({utilization_regular:.1f}% utilized)")
            
            print(f"  {'─'*96}")
            
            for s in room_surgeries:
                surg_id_str = str(s['surgery_id'])
                start_str = f"{int(s['start']//60):02d}:{int(s['start']%60):02d}"
                end_str = f"{int(s['end']//60):02d}:{int(s['end']%60):02d}"
                
                # Mark if surgery goes into overtime
                overtime_marker = " [OT]" if s['end'] > regular_hours else ""
                
                print(f"  • Surgery {surg_id_str:15s} ({s['surgery_name']:20s}) | "
                      f"{start_str}-{end_str} ({s['buffered_duration']:5.1f}m){overtime_marker} | "
                      f"Doctor: {s['doctor_name']:12s} | "
                      f"α={s['alpha']:.4f} | "
                      f"μ={s['duration_mean']:5.1f} σ={s['duration_std']:4.1f}")


def print_overtime_summary(overtime_info, instance, ot_cost_room, ot_cost_doc):
    """Print summary of overtime usage and costs."""
    print("\n" + "="*100)
    print("💰 OVERTIME SUMMARY")
    print("="*100)
    
    total_room_ot_min = 0
    total_doc_ot_min = 0
    
    # Room overtime
    print("\n🏥 Room Overtime:")
    print(f"{'─'*60}")
    if overtime_info['room']:
        for day_id in sorted(overtime_info['room'].keys()):
            day = next((d for d in instance['days'] if d['id'] == day_id), None)
            day_display = day.get('name', str(day_id)) if day else str(day_id)
            
            for room_id, ot_min in overtime_info['room'][day_id].items():
                room = next((r for r in instance['rooms'] if r['id'] == room_id), None)
                room_display = room.get('name', str(room_id)) if room else str(room_id)
                cost = ot_min * ot_cost_room
                total_room_ot_min += ot_min
                print(f"  {day_display:12s} | {room_display:8s} | {ot_min:6.1f} min | €{cost:7.2f}")
    else:
        print("  No room overtime used ✓")
    
    print(f"{'─'*60}")
    print(f"  {'TOTAL ROOM OT':>31s} | {total_room_ot_min:6.1f} min | €{total_room_ot_min*ot_cost_room:7.2f}")
    
    # Doctor overtime
    print(f"\n👨‍⚕️ Doctor Overtime:")
    print(f"{'─'*60}")
    if overtime_info['doctor']:
        for day_id in sorted(overtime_info['doctor'].keys()):
            day = next((d for d in instance['days'] if d['id'] == day_id), None)
            day_display = day.get('name', str(day_id)) if day else str(day_id)
            
            for doc_id, ot_min in overtime_info['doctor'][day_id].items():
                doctor = next((doc for doc in instance['doctors'] if doc['id'] == doc_id), None)
                doc_display = doctor.get('name', str(doc_id)) if doctor else str(doc_id)
                cost = ot_min * ot_cost_doc
                total_doc_ot_min += ot_min
                print(f"  {day_display:12s} | {doc_display:12s} | {ot_min:6.1f} min | €{cost:7.2f}")
    else:
        print("  No doctor overtime used ✓")
    
    print(f"{'─'*60}")
    print(f"  {'TOTAL DOCTOR OT':>35s} | {total_doc_ot_min:6.1f} min | €{total_doc_ot_min*ot_cost_doc:7.2f}")
    
    # Grand total
    total_ot_cost = total_room_ot_min * ot_cost_room + total_doc_ot_min * ot_cost_doc
    total_ot_hours = (total_room_ot_min + total_doc_ot_min) / 60
    
    print(f"\n{'='*60}")
    print(f"  GRAND TOTAL: {total_room_ot_min + total_doc_ot_min:.1f} min ({total_ot_hours:.2f} hours) | €{total_ot_cost:.2f}")
    print(f"{'='*60}")


def print_summary_statistics(schedule, instance, idle_time, epsilon_dict, overtime_info):
    """Print summary statistics including overtime."""
    print("\n" + "="*100)
    print("📊 SUMMARY STATISTICS")
    print("="*100)
    
    total_capacity = sum(d['H'] for d in instance['days']) * len(instance['rooms'])
    utilization = 100 * (1 - idle_time / total_capacity) if total_capacity > 0 else 0
    
    print(f"\n{'Overall Metrics':40s}")
    print(f"{'─'*60}")
    print(f"  Total surgeries scheduled: {len(schedule)}/{len(instance['surgeries'])}")
    print(f"  Total regular capacity: {total_capacity:,.0f} minutes")
    print(f"  Total used time: {total_capacity - idle_time:,.0f} minutes")
    print(f"  Total idle time (in regular hours): {idle_time:,.0f} minutes")
    print(f"  Regular hours utilization: {utilization:.2f}%")
    
    # Overtime metrics
    total_room_ot = sum(ot for day_ot in overtime_info['room'].values() for ot in day_ot.values())
    total_doc_ot = sum(ot for day_ot in overtime_info['doctor'].values() for ot in day_ot.values())
    
    if total_room_ot > 0 or total_doc_ot > 0:
        print(f"\n  💰 Overtime Usage:")
        print(f"    • Total room overtime: {total_room_ot:.1f} minutes")
        print(f"    • Total doctor overtime: {total_doc_ot:.1f} minutes")
    
    # Per-day statistics
    print(f"\n{'Per-Day Breakdown':40s}")
    print(f"{'─'*60}")
    
    by_day = {}
    for s in schedule:
        if s['day'] not in by_day:
            by_day[s['day']] = []
        by_day[s['day']].append(s)
    
    day_order = [d['id'] for d in instance['days']]
    
    for day_id in day_order:
        if day_id not in by_day:
            continue
        
        day = next(d for d in instance['days'] if d['id'] == day_id)
        surgeries = by_day[day_id]
        
        day_capacity = day['H'] * len(instance['rooms'])
        day_used = sum(s['buffered_duration'] for s in surgeries)
        day_util = 100 * day_used / day_capacity if day_capacity > 0 else 0
        day_alpha_sum = sum(s['alpha'] for s in surgeries)
        
        day_room_ot = sum(overtime_info['room'].get(day_id, {}).values())
        day_doc_ot = sum(overtime_info['doctor'].get(day_id, {}).values())
        
        if isinstance(epsilon_dict, dict):
            epsilon_val = epsilon_dict.get(day_id, 'N/A')
        else:
            epsilon_val = epsilon_dict
        
        day_display = day.get('name', str(day_id))
        
        ot_str = f" [OT: room={day_room_ot:.0f}, doc={day_doc_ot:.0f}]" if (day_room_ot > 0 or day_doc_ot > 0) else ""
        
        print(f"  {day_display:12s}: {len(surgeries):2d} surgeries | "
              f"{day_used:6.0f}/{day_capacity:6.0f} min | "
              f"{day_util:5.1f}% util | "
              f"Σα={day_alpha_sum:6.4f} (ε={epsilon_val:.4f}){ot_str}")
    
    # By specialty
    print(f"\n{'By Specialty':40s}")
    print(f"{'─'*60}")
    
    by_specialty = {}
    for s in schedule:
        spec = s['specialty']
        if spec not in by_specialty:
            by_specialty[spec] = []
        by_specialty[spec].append(s)
    
    for spec in sorted(by_specialty.keys()):
        surgeries = by_specialty[spec]
        avg_alpha = sum(s['alpha'] for s in surgeries) / len(surgeries)
        print(f"  {spec:15s}: {len(surgeries):2d} surgeries | Avg α = {avg_alpha:.4f}")
    
    # Alpha distribution
    print(f"\n{'Alpha Value Distribution':40s}")
    print(f"{'─'*60}")
    
    alpha_counts = {}
    for s in schedule:
        alpha = s['alpha']
        alpha_counts[alpha] = alpha_counts.get(alpha, 0) + 1
    
    for alpha in sorted(alpha_counts.keys()):
        count = alpha_counts[alpha]
        pct = 100 * count / len(schedule)
        bar = '█' * int(pct / 2)
        print(f"  α = {alpha:.4f}: {count:3d} surgeries ({pct:5.1f}%) {bar}")


def print_reliability_analysis(schedule, instance, alpha_choices, epsilon_dict):
    """Detailed reliability analysis using LOG-PRODUCT FORM."""
    print("\n" + "="*100)
    print("🎯 RELIABILITY ANALYSIS (Log-Product Form)")
    print("="*100)
    print("\nReliability Constraint: Σ ln(1-α_jt) ≥ ln(1-ε_t) for each day t")
    print("\nThis ensures the probability that NO surgery on day t exceeds its buffer")
    print("is at least (1-ε_t), i.e., at most ε_t probability of ANY overrun on day t.")
    print("\n⚠️ NOTE: Reliability is computed using REGULAR HOURS only (not including overtime)")
    
    by_day = {}
    for s in schedule:
        if s['day'] not in by_day:
            by_day[s['day']] = []
        by_day[s['day']].append(s)
    
    ln1ma = {a: math.log(1 - a) for a in alpha_choices}
    
    print(f"\n{'Day Analysis':40s}")
    print(f"{'─'*80}")
    print(f"{'Day':17s} {'Surgeries':>12s} {'Σ ln(1-α)':>14s} {'ln(1-ε)':>14s} {'Slack':>12s} {'Status':>10s}")
    print(f"{'─'*80}")
    
    day_order = [d['id'] for d in instance['days']]
    total_lhs = 0
    total_rhs = 0
    
    for day_id in day_order:
        if day_id not in by_day:
            continue
        day = next(d for d in instance['days'] if d['id'] == day_id)
        surgeries = by_day[day_id]
        
        lhs = sum(ln1ma[s['alpha']] for s in surgeries)
        
        if isinstance(epsilon_dict, dict):
            epsilon_val = epsilon_dict.get(day_id, 0.25)
        else:
            epsilon_val = epsilon_dict
        rhs = math.log(1 - epsilon_val)
        
        slack = lhs - rhs
        status = "✓ OK" if slack >= -1e-6 else "✗ VIOLATED"
        
        day_display = day.get('name', str(day_id))
        print(f"{day_display:17s} {len(surgeries):12d} {lhs:14.6f} {rhs:14.6f} "
              f"{slack:12.6f} {status:>10s}")
        
        total_lhs += lhs
        total_rhs += rhs
    
    print(f"{'─'*80}")
    print(f"{'TOTAL':17s} {len(schedule):12d} {total_lhs:14.6f} {total_rhs:14.6f} "
          f"{total_lhs - total_rhs:12.6f} {'─'*10}")
    
    print(f"\nInterpretation:")
    print(f"  • Each day's constraint: ∏(1-α_j) ≥ (1-ε) [product of reliabilities]")
    print(f"  • Log form: Σ ln(1-α_j) ≥ ln(1-ε) [sum of log-reliabilities]")
    print(f"  • Tighter ε → more negative ln(1-ε) → need smaller α values → more buffer")
    print(f"  • Current ε = {epsilon_dict if not isinstance(epsilon_dict, dict) else 'varies'}")


def generate_35_surgery_5_rooms_NO_SPECIALIZATION():
    """Generate 35 surgeries, 5 rooms, 6 doctors - NO SPECIALIZATION."""
    days = [
        {"id": "Monday", "H": 960, "name": "Monday"},
        {"id": "Tuesday", "H": 960, "name": "Tuesday"},
        {"id": "Wednesday", "H": 900, "name": "Wednesday"},
        {"id": "Thursday", "H": 900, "name": "Thursday"},
        {"id": "Friday", "H": 840, "name": "Friday"},
    ]
    
    rooms = [
        {"id": "OR1", "name": "OR1"},
        {"id": "OR2", "name": "OR2"},
        {"id": "OR3", "name": "OR3"},
        {"id": "OR4", "name": "OR4"},
        {"id": "OR5", "name": "OR5"},
    ]
    
    doctors = [
        {"id": "Doctor_1", "name": "Doctor_1", "daily_capacity": {
            "Monday": 900, "Tuesday": 900, "Wednesday": 850, "Thursday": 850, "Friday": 800}},
        {"id": "Doctor_2", "name": "Doctor_2", "daily_capacity": {
            "Monday": 900, "Tuesday": 900, "Wednesday": 850, "Thursday": 850, "Friday": 800}},
        {"id": "Doctor_3", "name": "Doctor_3", "daily_capacity": {
            "Monday": 900, "Tuesday": 900, "Wednesday": 850, "Thursday": 850, "Friday": 800}},
        {"id": "Doctor_4", "name": "Doctor_4", "daily_capacity": {
            "Monday": 850, "Tuesday": 850, "Wednesday": 800, "Thursday": 800, "Friday": 750}},
        {"id": "Doctor_5", "name": "Doctor_5", "daily_capacity": {
            "Monday": 850, "Tuesday": 850, "Wednesday": 800, "Thursday": 800, "Friday": 750}},
        {"id": "Doctor_6", "name": "Doctor_6", "daily_capacity": {
            "Monday": 850, "Tuesday": 850, "Wednesday": 800, "Thursday": 800, "Friday": 750}},
    ]
    
    surgeries = []
    all_combinations = []
    for room in rooms:
        for doctor in doctors:
            all_combinations.append(f"{room['id']}|{doctor['id']}")
    
    print(f"  All {len(doctors)} doctors can use all {len(rooms)} rooms: {len(all_combinations)} combinations per surgery")
    
    # 12 Small surgeries
    for i in range(1, 13):
        mu_sigma_dict = {}
        base_mu = 60 + i*5
        base_sigma = 18 + i*1
        
        for key in all_combinations:
            parts = key.split('|')
            doc_num = int(parts[1].split('_')[1])
            room_num = int(parts[0].replace('OR', ''))
            
            mu_sigma_dict[key] = {
                "mu": base_mu + (doc_num - 3) * 2 + (room_num - 3) * 1,
                "sigma": base_sigma + (doc_num - 3) * 0.5
            }
        
        surgeries.append({
            "id": f"Small_{i}",
            "name": f"Small_{i}",
            "specialty": "General",
            "mu": base_mu,
            "sigma": base_sigma,
            "mu_sigma": mu_sigma_dict
        })
    
    # 12 Medium surgeries
    for i in range(1, 13):
        mu_sigma_dict = {}
        base_mu = 150 + i*8
        base_sigma = 40 + i*1.5
        
        for key in all_combinations:
            parts = key.split('|')
            doc_num = int(parts[1].split('_')[1])
            room_num = int(parts[0].replace('OR', ''))
            
            mu_sigma_dict[key] = {
                "mu": base_mu + (doc_num - 3) * 3 + (room_num - 3) * 1.5,
                "sigma": base_sigma + (doc_num - 3) * 1
            }
        
        surgeries.append({
            "id": f"Medium_{i}",
            "name": f"Medium_{i}",
            "specialty": "General",
            "mu": base_mu,
            "sigma": base_sigma,
            "mu_sigma": mu_sigma_dict
        })
    
    # 11 Large surgeries
    for i in range(1, 12):
        mu_sigma_dict = {}
        base_mu = 280 + i*11
        base_sigma = 65 + i*2.5
        
        for key in all_combinations:
            parts = key.split('|')
            doc_num = int(parts[1].split('_')[1])
            room_num = int(parts[0].replace('OR', ''))
            
            mu_sigma_dict[key] = {
                "mu": base_mu + (doc_num - 3) * 4 + (room_num - 3) * 2,
                "sigma": base_sigma + (doc_num - 3) * 1.5
            }
        
        surgeries.append({
            "id": f"Large_{i}",
            "name": f"Large_{i}",
            "specialty": "General",
            "mu": base_mu,
            "sigma": base_sigma,
            "mu_sigma": mu_sigma_dict
        })
    
    return {"surgeries": surgeries, "days": days, "rooms": rooms, "doctors": doctors}


def print_problem_overview(instance):
    """Print overview of the problem instance."""
    print("\n" + "="*80)
    print("📋 PROBLEM INSTANCE")
    print("="*80)
    
    print(f"\n  Surgeries: {len(instance['surgeries'])}")
    print(f"  Days: {len(instance['days'])}")
    print(f"  Rooms: {len(instance['rooms'])}")
    print(f"  Doctors: {len(instance['doctors'])}")
    
    # Count by surgery type
    if instance['surgeries']:
        by_type = {}
        for s in instance['surgeries']:
            surg_id = s['id']
            if '_' in str(surg_id):
                stype = str(surg_id).split('_')[0]
                by_type[stype] = by_type.get(stype, 0) + 1
            elif 'specialty' in s:
                spec = s['specialty']
                by_type[spec] = by_type.get(spec, 0) + 1
        
        if by_type:
            print(f"\n  By type/specialty:")
            for stype, count in sorted(by_type.items()):
                print(f"    • {stype}: {count} surgeries")
    
    # Total capacity
    total_capacity = sum(d['H'] for d in instance['days']) * len(instance['rooms'])
    
    # Total demand
    total_demand = 0
    for s in instance['surgeries']:
        if 'mu' in s:
            total_demand += s['mu']
        elif 'duration_mean' in s:
            total_demand += s['duration_mean']
    
    print(f"\n  Capacity Analysis:")
    print(f"    • Total room capacity: {total_capacity:,} minutes")
    if total_demand > 0:
        print(f"    • Total mean duration: {total_demand:,.0f} minutes")
        print(f"    • Capacity utilization (mean): {100 * total_demand / total_capacity:.1f}%")


def main():
    """Main execution function."""
    print("\n" + "="*80)
    print("  SURGERY SCHEDULING WITH OVERTIME PENALTIES")
    print("="*80)
    print(f"  Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*80)
    
    print("\n📋 MODEL FEATURES:")
    print("  ✓ SOFT capacity constraints with overtime penalties")
    print("  ✓ Overtime variables for rooms and doctors")
    print("  ✓ Hard caps on maximum overtime")
    print("  ✓ Overtime costs in objective function")
    print("  ✓ Reliability constraints use regular hours (not OT)")
    print("  ✓ Start time variables for temporal scheduling")
    print("  ✓ Ordering variables (u_room, u_doc) to prevent overlaps")
    print("  ✓ Room-doctor start time synchronization")
    print("  ✓ CORRECT Cantelli formula: δ = μ + σ*sqrt((1-α)/α)")
    print("  ✓ Log-based reliability: Σ log(1-α_ᵢ) ≥ log(1-ε)")
    print("="*80)
    
    # Configuration
    ALPHA_CHOICES = [0.005, 0.008, 0.01, 0.02, 0.03, 0.05, 0.10]
    FIXED_EPSILON = 0.25
    
    # Overtime parameters
    OT_COST_ROOM = 3.0   # €/min for room overtime
    OT_COST_DOC = 1.5    # €/min for doctor overtime
    MAX_OT_ROOM = 120    # Max 2 hours per room per day
    MAX_OT_DOC = 60      # Max 1 hour per doctor per day
    
    TIME_LIMIT = 900  # 15 minutes
    MIP_GAP = 0.15    # 15% gap
    
    print("\n⚙️ CONFIGURATION:")
    print(f"   Alpha choices: {ALPHA_CHOICES}")
    print(f"   Epsilon (reliability): {FIXED_EPSILON}")
    print(f"   Room overtime cost: €{OT_COST_ROOM:.2f}/min")
    print(f"   Doctor overtime cost: €{OT_COST_DOC:.2f}/min")
    print(f"   Max room overtime: {MAX_OT_ROOM} min/day")
    print(f"   Max doctor overtime: {MAX_OT_DOC} min/day")
    
    # Create instance
    print("\nCreating problem instance (NO SPECIALIZATION)...")
    instance = generate_35_surgery_5_rooms_NO_SPECIALIZATION()
    print(f"  ✓ 35 surgeries: 12 small, 12 medium, 11 large")
    print(f"  ✓ 5 rooms, 6 doctors")
    print(f"  ✓ All doctors can do all surgeries in all rooms")
    
    # Print problem overview
    print_problem_overview(instance)
    
    # Build model
    print("\n" + "="*80)
    print("🔨 BUILDING MODEL WITH OVERTIME")
    print("="*80)
    model, durations, surg_dict, day_dict, doc_dict, alpha_choices = build_model_with_overtime(
        instance, ALPHA_CHOICES, FIXED_EPSILON,
        ot_cost_room=OT_COST_ROOM,
        ot_cost_doc=OT_COST_DOC,
        max_ot_room=MAX_OT_ROOM,
        max_ot_doc=MAX_OT_DOC
    )
    
    # Solve model
    print("\n" + "="*80)
    print("🚀 SOLVING MODEL")
    print("="*80)
    results, solve_time = solve_model(model, TIME_LIMIT, MIP_GAP)
    
    # Check results
    term = results.solver.termination_condition
    
    if term in [TerminationCondition.optimal, TerminationCondition.maxTimeLimit]:
        # Extract solution
        schedule, idle, overtime_info = extract_schedule_with_overtime(
            model, instance, durations, alpha_choices
        )
        
        if len(schedule) == len(instance['surgeries']):
            print("\n✅ SOLUTION FOUND!")
            print(f"   All {len(schedule)} surgeries scheduled")
            
            # Print detailed schedule with overtime
            print_schedule_with_overtime(schedule, instance, FIXED_EPSILON, 
                                        overtime_info, OT_COST_ROOM, OT_COST_DOC)
            
            # Print overtime summary
            print_overtime_summary(overtime_info, instance, OT_COST_ROOM, OT_COST_DOC)
            
            # Print summary statistics
            print_summary_statistics(schedule, instance, idle, FIXED_EPSILON, overtime_info)
            
            # Print reliability analysis
            print_reliability_analysis(schedule, instance, alpha_choices, FIXED_EPSILON)
            
            # Final summary
            total_cap = sum(d['H'] for d in instance['days']) * len(instance['rooms'])
            utilization = 100 * (1 - idle / total_cap) if total_cap > 0 else 0
            
            total_room_ot = sum(ot for day_ot in overtime_info['room'].values() for ot in day_ot.values())
            total_doc_ot = sum(ot for day_ot in overtime_info['doctor'].values() for ot in day_ot.values())
            total_ot_cost = total_room_ot * OT_COST_ROOM + total_doc_ot * OT_COST_DOC
            
            print(f"\n{'='*80}")
            print(f"✅ OPTIMIZATION COMPLETE!")
            print(f"{'='*80}")
            print(f"\n📊 FINAL RESULTS:")
            print(f"   Objective value: {pyo.value(model.obj):.2f}")
            print(f"   Regular hours utilization: {utilization:.1f}%")
            print(f"   Total overtime: {total_room_ot + total_doc_ot:.1f} min")
            print(f"   Total overtime cost: €{total_ot_cost:.2f}")
            print(f"   Solve time: {solve_time:.1f}s ({solve_time/60:.1f} min)")
            print(f"   Termination: {term}")
            print(f"   Epsilon (fixed): {FIXED_EPSILON:.4f}")
            print(f"\n{'='*80}")
        else:
            print(f"\n❌ PARTIAL SOLUTION")
            print(f"   Only {len(schedule)}/{len(instance['surgeries'])} surgeries scheduled")
    else:
        print(f"\n❌ SOLVE FAILED")
        print(f"   Termination: {term}")


if __name__ == "__main__":
    main()


  SURGERY SCHEDULING WITH OVERTIME PENALTIES
  Started: 2025-11-01 11:53:02

📋 MODEL FEATURES:
  ✓ SOFT capacity constraints with overtime penalties
  ✓ Overtime variables for rooms and doctors
  ✓ Hard caps on maximum overtime
  ✓ Overtime costs in objective function
  ✓ Reliability constraints use regular hours (not OT)
  ✓ Start time variables for temporal scheduling
  ✓ Ordering variables (u_room, u_doc) to prevent overlaps
  ✓ Room-doctor start time synchronization
  ✓ CORRECT Cantelli formula: δ = μ + σ*sqrt((1-α)/α)
  ✓ Log-based reliability: Σ log(1-α_ᵢ) ≥ log(1-ε)

⚙️ CONFIGURATION:
   Alpha choices: [0.005, 0.008, 0.01, 0.02, 0.03, 0.05, 0.1]
   Epsilon (reliability): 0.25
   Room overtime cost: €3.00/min
   Doctor overtime cost: €1.50/min
   Max room overtime: 120 min/day
   Max doctor overtime: 60 min/day

Creating problem instance (NO SPECIALIZATION)...
  All 6 doctors can use all 5 rooms: 30 combinations per surgery
  ✓ 35 surgeries: 12 small, 12 medium, 11 large
  ✓ 5 r